In [1]:
# Uncomment and run to reload libs
# import importlib
# import utils
# importlib.reload(utils)
# import utils.miscellaneous
# importlib.reload(utils.miscellaneous)


from itertools import combinations

import pandas as pd

from utils import (
    logging,
    COMBO_MIN_NUM,
    RECOMMENDED_MUTATIONS_FILE,
    EXISTING_COMBO_FILE,
    FUTURE_COMBO_FILE,
)
from utils.reader import AminoAcidComboReader
from utils.miscellaneous import sort_mut_by_pos


In [2]:
pred_scores = pd.read_csv(RECOMMENDED_MUTATIONS_FILE)

aa_combo_reader: AminoAcidComboReader = AminoAcidComboReader()
training_data = aa_combo_reader.before_date_data()
test_data = aa_combo_reader.after_date_data()


In [3]:
existing_combo = set()

ac_group: pd.DataFrame
for ac, ac_group in training_data.groupby("Accession", sort=False):
    for i in combinations(ac_group["AA"].values, COMBO_MIN_NUM):
        existing_combo.add(frozenset(i))
        # break

len(existing_combo)


2460040

In [4]:
sort_mut_by_pos(
    pd.DataFrame(existing_combo).rename(columns={0: "Accession", 1: "Mut"}),
    "Accession",
    "Mut"
).to_csv(EXISTING_COMBO_FILE, index=False)


In [5]:
future_combo = []
existing_future_combo = set()
ac_group: pd.DataFrame
c_date_group: pd.DataFrame
for c_date, c_date_group in test_data.groupby("Date", sort=False):
    logging.info(f"Future date: {c_date}")
    c_date_combo = set()
    for ac, ac_group in c_date_group.groupby("Accession", sort=False):
        for i in combinations(ac_group["AA"].values, COMBO_MIN_NUM):
            c_date_combo.add(frozenset(i))

    # Remove combo in the training data
    c_date_combo = c_date_combo.difference(existing_combo)
    # Reomve combo in the previous dates
    c_date_combo = c_date_combo.difference(existing_future_combo)
    existing_future_combo = existing_future_combo.union(c_date_combo)

    if len(c_date_combo):
        c_date_combo = pd.DataFrame(c_date_combo)
        c_date_combo = c_date_combo.rename(columns={0: "Accession", 1: "Mut"})
        c_date_combo = sort_mut_by_pos(c_date_combo, "Accession", "Mut")
        c_date_combo["Date"] = c_date
        future_combo.append(c_date_combo)


In [ ]:
pd.concat(future_combo).to_csv(FUTURE_COMBO_FILE, index=False)